## 作業目標: 了解斷詞演算法的背後計算

### 根據課程講述的內容, 請計算出下列剩餘所有情況的
若有一個人連續觀察到三天水草都是乾燥的(Dry), 則這三天的天氣機率為何？(可參考講義第13頁)
(Hint: 共有8種可能機率)

```python
states = ('sunny', 'rainy')
observations = ('dry', 'damp', 'soggy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}
```

In [1]:
observations = ('dry', 'dry', 'dry') #實際上觀察到的狀態為dry, dry, dry
states = ('sunny', 'rainy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}

In [2]:
# 觀察狀態 = 'dry', 'dry', 'dry'
# Sunny, Sunny, Sunny: 0.4*(0.6)*0.6*(0.6)*0.6*(0.6) = 0.031104
# Rainy, Sunny, Sunny: 0.6*(0.1)*0.3*(0.6)*0.6*(0.6) = 0.003888
for s1 in states:
    for s2 in states:
        for s3 in states:
            print(f'{s1.capitalize()}, {s2.capitalize()}, {s3.capitalize()}: ', end='')
            print(f'{start_probability[s1]}*{emission_probatility[s1]["dry"]}*', end='')
            print(f'{transition_probability[s1][s2]}*{emission_probatility[s2]["dry"]}*', end='')
            print(f'{transition_probability[s2][s3]}*{emission_probatility[s3]["dry"]} = ', end='')
            prob = start_probability[s1]*emission_probatility[s1]["dry"]*\
                   transition_probability[s1][s2]*emission_probatility[s2]["dry"]*\
                   transition_probability[s2][s3]*emission_probatility[s3]["dry"]
            print(f'{prob:.6f}')

# 最大機率為: Sunny, Sunny, Sunny

Sunny, Sunny, Sunny: 0.4*0.6*0.6*0.6*0.6*0.6 = 0.031104
Sunny, Sunny, Rainy: 0.4*0.6*0.6*0.6*0.4*0.1 = 0.003456
Sunny, Rainy, Sunny: 0.4*0.6*0.4*0.1*0.3*0.6 = 0.001728
Sunny, Rainy, Rainy: 0.4*0.6*0.4*0.1*0.7*0.1 = 0.000672
Rainy, Sunny, Sunny: 0.6*0.1*0.3*0.6*0.6*0.6 = 0.003888
Rainy, Sunny, Rainy: 0.6*0.1*0.3*0.6*0.4*0.1 = 0.000432
Rainy, Rainy, Sunny: 0.6*0.1*0.7*0.1*0.3*0.6 = 0.000756
Rainy, Rainy, Rainy: 0.6*0.1*0.7*0.1*0.7*0.1 = 0.000294


### 根據上述條件, 寫出Viterbi應用程式

In [3]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = []
    for i in range(len(obs)):
        for state in states:
            
            # 第一天的話為發射*開始，其餘為發射*轉換*前一次機率
            prob =  emit_p[state][obs[0]] * (trans_p[V[i-1][0]][state] * V[i-1][1] if i else start_p[state])
            
            # 當天第一次迭代新增狀態與機率，其餘為比較機率
            if len(V) > i:
                if prob > V[i][1]:
                    V[i] = [state, prob]
            else:
                V.append([state, prob])
    return (V[-1][-1], [i[0] for i in V]) # prob, path

In [4]:
result = viterbi(observations, states, start_probability, transition_probability, emission_probatility)
print(result)

(0.031103999999999993, ['sunny', 'sunny', 'sunny'])
